In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en" data-bs-theme="dark">

<head>
    <title>"Who wants to Bee a programmer?"</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <!-- MDB -->
    <link href="https://cdnjs.cloudflare.com/ajax/libs/mdb-ui-kit/6.3.0/mdb.dark.min.css" rel="stylesheet" />
    <style> {{styles}} </style>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/css/bootstrap.min.css" rel="stylesheet"
        integrity="sha384-KK94CHFLLe+nY2dmCWGMq91rCGa5gtU4mk92HdvYe+M/SXH301p5ILy+dN9+nJOZ" crossorigin="anonymous">
    <script src="https://www.gstatic.com/firebasejs/8.6.1/firebase-app.js"></script>
    <script src="https://www.gstatic.com/firebasejs/8.6.1/firebase-database.js"></script>
    <style>
        .my-btn {
            text-transform: none;
            /* Set text transformation to none */
        }
    </style>
</head>

<body>
    <div id="header"></div>
    <div id="game"></div>
    <div class="container align-items-center px-5 py-11">
        <canvas style="display: none; max-height:300px!important;" id="chartContainer"></canvas>
    </div>
    <script src="https://code.jquery.com/jquery-3.7.0.min.js"
        integrity="sha256-2Pmvv0kuTBOenSvLm6bvfBSSHrUJ+3A7x6P5Ebd07/g=" crossorigin="anonymous"></script>
    <!-- MDB -->
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/mdb-ui-kit/6.3.0/mdb.min.js"></script>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/js/bootstrap.bundle.min.js"
        integrity="sha384-ENjdO4Dr2bkBIFxQpeoTz1HIcje39Wm4jDKdf19U8gI4ddQ3GYNS7NTKfAdVQSZe"
        crossorigin="anonymous"></script>
    <script> {{script}} </script>
    <script src="index.js"></script>
</body>

</html>



Writing index.html


## For some reason data-bs-theme="dark" does not work in colab so I set it manually in the file bellow

In [ ]:
%%writefile styles.css
 body {
    color-scheme: dark;
    --bs-body-color: #adb5bd;
    --bs-body-color-rgb: 173,181,189;
    --bs-body-bg: #212529;
    --bs-body-bg-rgb: 33,37,41;
    --bs-emphasis-color: #fff;
    --bs-emphasis-color-rgb: 255,255,255;
    --bs-secondary-color: rgba(173, 181, 189, 0.75);
    --bs-secondary-color-rgb: 173,181,189;
    --bs-secondary-bg: #343a40;
    --bs-secondary-bg-rgb: 52,58,64;
    --bs-tertiary-color: rgba(173, 181, 189, 0.5);
    --bs-tertiary-color-rgb: 173,181,189;
    --bs-tertiary-bg: #2b3035;
    --bs-tertiary-bg-rgb: 43,48,53;
    --bs-primary-text-emphasis: #6ea8fe;
    --bs-secondary-text-emphasis: #a7acb1;
    --bs-success-text-emphasis: #75b798;
    --bs-info-text-emphasis: #6edff6;
    --bs-warning-text-emphasis: #ffda6a;
    --bs-danger-text-emphasis: #ea868f;
    --bs-light-text-emphasis: #f8f9fa;
    --bs-dark-text-emphasis: #dee2e6;
    --bs-primary-bg-subtle: #031633;
    --bs-secondary-bg-subtle: #161719;
    --bs-success-bg-subtle: #051b11;
    --bs-info-bg-subtle: #032830;
    --bs-warning-bg-subtle: #332701;
    --bs-danger-bg-subtle: #2c0b0e;
    --bs-light-bg-subtle: #343a40;
    --bs-dark-bg-subtle: #1a1d20;
    --bs-primary-border-subtle: #084298;
    --bs-secondary-border-subtle: #41464b;
    --bs-success-border-subtle: #0f5132;
    --bs-info-border-subtle: #087990;
    --bs-warning-border-subtle: #997404;
    --bs-danger-border-subtle: #842029;
    --bs-light-border-subtle: #495057;
    --bs-dark-border-subtle: #343a40;
    --bs-link-color: #6ea8fe;
    --bs-link-hover-color: #8bb9fe;
    --bs-link-color-rgb: 110,168,254;
    --bs-link-hover-color-rgb: 139,185,254;
    --bs-code-color: #e685b5;
    --bs-border-color: #495057;
    --bs-border-color-translucent: rgba(255, 255, 255, 0.15);
    --bs-form-valid-color: #75b798;
    --bs-form-valid-border-color: #75b798;
    --bs-form-invalid-color: #ea868f;
    --bs-form-invalid-border-color: #ea868f;
}


  #chartContainer .chartjs-legend span {
    color: red !important; /* Change the color of the legend label */
  }

  #chartContainer .chartjs-axis-label {
    color: blue !important; /* Change the color of the y-axis labels */
  }


Writing styles.css


In [ ]:
%%writefile index.js
let users = [];
let allQuestions = [];
let questions = [];

firebaseConfig = {
  "databaseURL": "https://tir6db-default-rtdb.firebaseio.com/"
}

firebase.initializeApp(firebaseConfig);
database = firebase.database()

/**
 * Retrieves data from the specified folder in the database and invokes a callback function with the retrieved data.
 */
function getFromDataBase(folder, callback) {
  database.ref(folder).once("value")
    .then(function(snapshot) {
      var value = snapshot.val();
      data = []
      for (var id in value) {
        val = value[id]
        val.id = id
        data.push(val);
      }
      callback(data);
    });
}

getFromDataBase("Users", (data) => { users = data; });
getFromDataBase("Questions", (data) => {
  data.map((element) => {
    allQuestions.push({
      correct: element.Correct - 1,
      answers: [element.Option1, element.Option2, element.Option3, element.Option4],
      question: element.Question,
    })
  })
});


var gameContainer = document.getElementById("game");
var loggedInUser = null
let chart;

function modalClose() {
  // Find the modal sheet element by its ID and remove it from the DOM
  document.getElementById("modalSheet").remove()
}

function modalShow(title, information) {
  // removintg previous modal
  var modal_element = document.getElementById("loginModal");
  if (modal_element) {
    modal_element.remove();
  }

  // creating new modal
  var modal =
    `
<div class="modal fade" id="loginModal" tabindex="-1" aria-labelledby="exampleModalLabel" aria-hidden="true">
  <div class="modal-dialog">
    <div class="modal-content">
      <div class="modal-header">
          <h1 class="modal-title fs-5">${title}</h1>
          <button type="button" class="btn-close" data-bs-dismiss="modal" aria-label="Close"></button>
        </div>
        <div class="modal-body py-0">
          <p>${information}</p>
        </div>
        <div class="modal-footer flex-column align-items-stretch w-100 gap-2 pb-3 border-top-0">
          <button type="button" class="btn btn-lg btn-primary" data-bs-dismiss="modal">OK</button>
        </div>
      </div>
    </div>
  </div>
  `
  gameContainer.innerHTML += modal;
}


function login() {
  $("#header").html("")
  loggedInUser = null
  var loginhtml = `
<div class="container col-xl-10 col-xxl-8 px-4 py-5">
    <div class="row align-items-center g-lg-5 py-5">
      <div class="col-lg-7 text-center text-lg-start">
        <h1 class="display-4 fw-bold lh-1 text-body-emphasis mb-3">Who Wants to
<img src="https://www.svgrepo.com/show/115092/bee.svg" alt="Bee Icon" width="70" height="70">
        a Programmer</h1>
        <p class="col-lg-10 fs-4"> This is a trivia game about programming in python</p>
      </div>

      <div class="col-md-10 mx-auto col-lg-5 text-center">
        <div class="p-4 p-md-5 border rounded-3 bg-body-tertiary">
          <div class="form-floating mb-3">
            <input type="text" class="form-control" id="username" placeholder="Username" data-ddg-inputtype="credentials.username" data-ddg-autofill="true" style="background-size: auto 26px !important; background-position: right center !important; background-repeat: no-repeat !important; background-origin: content-box !important; background-image: url(&quot;chrome-extension://bkdgflcldnnnapblkhphbgpggdiikppg/img/logo-small-grayscale.svg&quot;) !important; transition: background 0s ease 0s !important;">
            <label for="floatingInput">Username</label>
          </div>
          <div class="form-floating mb-3">
            <input type="password" class="form-control" id="password" placeholder="Password" data-ddg-inputtype="credentials.password">
            <label for="floatingPassword">Password</label>
          </div>
          <div id="confirm-password" class="form-floating mb-3" style="display: none;">
            <input type="password" id="conf-password" class="form-control"  placeholder="Password Confirmation" data-ddg-inputtype="credentials.password">
            <label for="floatingPassword">Password Confirmation</label>
          </div>
          <button class="w-100 btn btn-lg btn-primary my-btn" id="login-button" type="submit" onclick='check_login()'>Login</button>
          <hr class="my-4">
          <small id="login-text" class="text-body-secondary">Enter your Username and Password for login.</small>
<br>
            <button id="login-link" class="btn btn-link rounded-pill px-3" type="button" onclick="loginToggle()" >Sign Up</button>
        </div>
      </div>
    </div>
  </div>
`

  gameContainer.innerHTML = loginhtml
}
login();

function loginToggle() {

  var isLoggin = document.getElementById("login-button").innerText == "Login";
  if (isLoggin) {
    document.getElementById("login-button").innerText = "Sign Up";
    document.getElementById("login-text").innerText = "Enter your Username and Password for Sign Up.";
    document.getElementById("login-link").innerText = "Login";
    $("#confirm-password").slideToggle(1000);
  }
  else {
    document.getElementById("login-button").innerText = "Login";
    document.getElementById("login-text").innerTextext = "Enter your Username and Password for Login.";
    document.getElementById("login-link").innerText = "Sign Up";
    $("#confirm-password").slideToggle(1000);
  }
}

function check_signup() {
  var username = document.getElementById("username").value;
  var password = document.getElementById("password").value;
  var confirm_password = document.getElementById("conf-password").value;

  if (password != confirm_password) {
    modalShow("Error Sign Up", "Password and Confirm Password do not match.")
    return bootstrap.Modal.getOrCreateInstance(document.getElementById("loginModal")).show()
  }

  for (var i = 0; i < users.length; i++) {
    if (users[i]["userName"] === username) {

      modalShow("Error Sign Up", "This username is already exits in our database, Please choose another one.")
      return bootstrap.Modal.getOrCreateInstance(document.getElementById("loginModal")).show()
    }
  }

  var newUser = {
    "userName": username,
    "Password": password,
  }

  var newUserRef = database.ref("Users").push(newUser);
  newUserRef.then((snapshot) => {
    login();
    newUser.id = snapshot.key
    users.push(newUser);
    modalShow("Sign Up Successfull", "Hey " + username + ", You have successfully signed up.")
    bootstrap.Modal.getOrCreateInstance(document.getElementById("loginModal")).show()
  }).catch((error) => {
    login();
    modalShow("Sign Up Error", "Try again later.\n Error details: " + error)
    bootstrap.Modal.getOrCreateInstance(document.getElementById("loginModal")).show()
    console.log("Error: ", error);
  })
}

function check_login() {
  if (document.getElementById("login-button").innerText != "Login") {
    return check_signup();
  }
  var username = document.getElementById("username").value;
  var password = document.getElementById("password").value;

  for (var i = 0; i < users.length; i++) {
    if (users[i]["userName"] === username && users[i]["Password"] === password) {
      modalShow("Login Successful", "Welcome " + username + "!  \n you are now logged in and ready to play.")
      loggedInUser = users[i];
      createLineChart(loggedInUser.scores);
      homeScreen();
      return bootstrap.Modal.getOrCreateInstance(document.getElementById("loginModal")).show()
    }
  }
  modalShow("Login Erorr", "Username or password is incorrect, Please try again.")
  return bootstrap.Modal.getOrCreateInstance(document.getElementById("loginModal")).show()
}

function homeScreen() {
  let username = loggedInUser.userName;

  $("#header").html(`
        <h5 class="text-body-emphasis mb-3">
        <img class="mx-2" src='https://cdn.icon-icons.com/icons2/2643/PNG/512/male_boy_person_people_avatar_icon_159358.png' style="width:35px;height:35px;">
        ${username}
        <button type='submit' class='btn btn-link my-btn' onclick='login()'>Log Out</button>
        </h5>`)


  var enterHtml =
    `
<div class="container px-4 py-5">
    <div class="align-items-center py-5">
      <div class="text-center">
    <div class='homescreen'>
        <img src='https://cdn.icon-icons.com/icons2/2643/PNG/512/male_boy_person_people_avatar_icon_159358.png' style="width:100px;height:100px;">
        <h1 class="display-4 fw-bold lh-1 text-body-emphasis mb-3">Welcome ${username}</h1>

        <button type='submit' class='btn btn-primary' onclick='startGame()'>Start Game</button><br>
        <button type='submit' class='btn btn-secondary my-2' onclick='helpPage()'>How to Play</button>
        </div>
      </div>
    </div>
  </div>`;
  gameContainer.innerHTML = enterHtml;
}

function startGame() {
  // Reset the variables for a new game
  currentQuestion = 0; // Index of the current question
  score = 0; // Player's score
  userAnswers = []; // Array to store the player's answers

  // Shuffle the array of all questions
  allQuestions = shuffleArray(allQuestions);

  // Select the first 5 questions for the game
  questions = allQuestions.slice(0, 5);

  // Display the questions
  showQuestions();
}

/**
 * This function displays the question and its correct answer at the result page,
 * allowing the user to learn from their mistakes.
 */
function showAnswer(index) {
  // shows the question and answer at the result page so the user can learn from their mistakes
  var answersDiv = $("#show-answers");
  var q = questions[index];
  var newhtml = `<h4>${q.question}</h4><h5>${q.answers[q.correct]}</h5>`;
  if (answersDiv.is(":visible")) {
    answersDiv.slideToggle(() => {
      answersDiv.html(newhtml);
      answersDiv.slideToggle();
    });
  }
  else {
    answersDiv.html(newhtml);
    answersDiv.slideToggle();
  }
}


function showAnswerExample(index) {
  // shows the question and answer at the result page so the user can learn from their mistakes
  var answersDiv = $("#show-answers");
  var newhtml = `<h4>Question ${index}</h4><h5>Correct Answer ${index}</h5>`;
  if (answersDiv.is(":visible")) {
    answersDiv.slideToggle(() => {
      answersDiv.html(newhtml);
      answersDiv.slideToggle();
    });
  }
  else {
    answersDiv.html(newhtml);
    answersDiv.slideToggle();
  }
}

/**
 * This function displays the current question and answer options on the game screen.
 * If there are no more questions, it ask to finish the game.
 */
function showQuestions() {
  // check if there are no more questions, call the endGame function
  if (currentQuestion >= questions.length) {
    return endGame()
  }

  var question = questions[currentQuestion];
  var answers = question['answers'];
  var correct = question['correct'];
  var answersHtml = ""
  answers.forEach((element, index) =>
    answersHtml = answersHtml + `<div class="col-12 col-lg-6 grid"><button style="height: 100%;" type='submit' class='btn btn-primary row-12 col-12 btn-lg btn-block answer my-btn' id='${index}' onclick="checkAnswer('${index}',${correct})">${element}</button></div>`);

  var progress = ""
  questions.forEach((element, index) => {
    var cls = "secondary";
    if (index < userAnswers.length) {
      cls = (userAnswers[index]) ? "success" : "danger";
    }
    if (currentQuestion == index) {
      cls = "light";
    }
    progress = progress + `<button type='submit' class='btn btn-${cls} my-1 mx-1' id='progress${index}'>${index + 1}</button>`;

  });

  buttons = document.getElementsByClassName("answer");

  var gameHtml =
    `

    <div class="container px-4 py-5">
    <div class="align-items-center py-5 text-center">
      <h1 class="fs-1 fw-bold lh-2 text-body-emphasis mb-3">${question['question']}</h1>
      <div class='answers row gx-3 gy-3'>${answersHtml}</div>
      <div class = 'progress-buttons mt-4 mb-2'>
        ${progress}
      </div>
      <h4 class='score'>Score:${score}</h4>
    </div>
    </div>
    `;
  $("#game").fadeToggle(() => { gameContainer.innerHTML = gameHtml; $("#game").fadeToggle() });
}

/**
 * This function use to display the help page with information about the game
 */
function helpPage() {

  let questionExample = `<h1 class="fs-1 fw-bold lh-2 text-body-emphasis mb-3">The Question?</h1>
      <div class='answers row gx-3 gy-3'>
      <div class="col-12 col-lg-6 grid"><button style="height: 100%;" type='submit' class='btn btn-primary row-12 col-12 btn-lg btn-block answer my-btn' >Answer A</button></div>
      <div class="col-12 col-lg-6 grid"><button style="height: 100%;" type='submit' class='btn btn-primary row-12 col-12 btn-lg btn-block answer my-btn' >Answer B</button></div>
      <div class="col-12 col-lg-6 grid"><button style="height: 100%;" type='submit' class='btn btn-primary row-12 col-12 btn-lg btn-block answer my-btn' >Answer C</button></div>
      <div class="col-12 col-lg-6 grid"><button style="height: 100%;" type='submit' class='btn btn-primary row-12 col-12 btn-lg btn-block answer my-btn' >Answer D</button></div>
      </div>`

  let feedbackCorrect = `<button type='submit' class='my-3 btn btn-success btn-lg my-btn' >Correct Answer</button><br>
`

  let feedbackWrong = `<button type='submit' class='my-3 btn btn-danger btn-lg my-btn' >Wrong Answer</button><br>
`

  let feedbackProgress = `
  <button type='submit' class='btn btn-success my-1 mx-1' >1</button>
  <button type='submit' class='btn btn-danger my-1 mx-1' >2</button>
  <button type='submit' class='btn btn-light my-1 mx-1' >3</button>
  `
  let progressChart = "<canvas id='exampleChart' style='height: 80px; !important'></div>"
  let answersCorrect = `<button type='submit' class='btn btn-danger my-2 mx-1' id='progress1' onclick="showAnswerExample(1)">1</button>
    <button type='submit' class='btn btn-success my-2 mx-1' id='progress2' onclick="showAnswerExample(2)">2</button>
    <button type='submit' class='btn btn-danger my-2 mx-1' id='progress3' onclick="showAnswerExample(3)">3</button>`;

  helpHtml = `
  <div class="container px-4 py-5">
    <div class="align-items-center py-5">
      <div class="text-center">

    <!-- Tabs navs -->
<ul class="nav nav-tabs nav-justified mb-3" id="ex1" role="tablist">
  <li class="nav-item" role="presentation">
    <a
      class="nav-link active"
      id="ex3-tab-1"
      data-mdb-toggle="tab"
      href="#ex3-tabs-1"
      role="tab"
      aria-controls="ex3-tabs-1"
      aria-selected="true"
      >Answer Questions</a
    >
  </li>
  <li class="nav-item" role="presentation">
    <a
      class="nav-link"
      id="ex3-tab-2"
      data-mdb-toggle="tab"
      href="#ex3-tabs-2"
      role="tab"
      aria-controls="ex3-tabs-2"
      aria-selected="false"
      >Feedback </a
    >
  </li>
  <li class="nav-item" role="presentation">
    <a
      class="nav-link"
      id="ex3-tab-3"
      data-mdb-toggle="tab"
      href="#ex3-tabs-3"
      role="tab"
      aria-controls="ex3-tabs-3"
      aria-selected="false"
      >Review Your Game</a
    >
  </li>
</ul>
<!-- Tabs navs -->

<!-- Tabs content -->
<div class="tab-content" id="ex2-content">
  <div
    class="tab-pane fade show active"
    id="ex3-tabs-1"
    role="tabpanel"
    aria-labelledby="ex3-tab-1"
  >
    <h2>The Game</h2>
    ${questionExample}
    <p class="my-2">This is an Example of a Question. You will be prompt a question and 4 possible answers, you need to choose one of the answers.</p>
  </div>
  <div
    class="tab-pane fade"
    id="ex3-tabs-2"
    role="tabpanel"
    aria-labelledby="ex3-tab-2"
  ><div class="text-center align-items-center">
    <h2>Game Feedback</h2>
    <h5>Correct Answer:</h5>
    ${feedbackCorrect}
    <h5>Wrong Answer:</h5>
    ${feedbackWrong}
    <h5>You can see your progress bellow:</h5>
    ${feedbackProgress}
    <p>Green: correct answer, Red: wrong answer and White: current question.</p>
</div>

  </div>
  <div
    class="tab-pane fade"
    id="ex3-tabs-3"
    role="tabpanel"
    aria-labelledby="ex3-tab-3"
  >
    <h2>Review The Game</h2>
    <h5>View the Correct Answers:</h5>
    <p>In the end screen you can click on these buttons to reveal the correct answers.</p>
    ${answersCorrect}
    <div id="show-answers"></div>
    <h5>Progress Chart:</h5>
    <p>Show you the progress in your last games (Up to 15 games).</p>
    ${progressChart}

  </div>
</div>
<!-- Tabs content -->
      <div class="align-items-center py-2 text-center">
      <button class='start-btn btn btn-secondary my-4' onclick='homeScreen()'>Back</button>
      </div>

      </div>
    </div>
  </div>`;

  gameContainer.innerHTML = helpHtml
  createExampleChart()

}

/**
 * ends the game and displays the final score, progress, and top score board
 * and also updates the logged-in user's scores with the current game score.
 */
function endGame() {

  if (loggedInUser.scores == undefined) {
    loggedInUser.scores = []
  }
  loggedInUser.scores.push([new Date().toLocaleString(), score]);
  userRef = database.ref("Users/" + loggedInUser.id);
  console.log(loggedInUser.id)

  userRef.update({
    scores: loggedInUser.scores
  })



  var resultText = "You got " + score + " out of " + questions.length + " questions right!";

  var progress = ""
  questions.forEach((element, index) => {
    var cls = "secondary";
    if (index < userAnswers.length) {
      cls = (userAnswers[index]) ? "success" : "danger";
    }
    progress = progress + `<button type='submit' class='btn btn-${cls} my-2 mx-1' id='progress${index}' onclick="showAnswer(${index})">${index + 1}</button>`;
  });

  var gameHtml = `
    <div class="container px-5 py-5">
    <div class="align-items-center py-6 text-center">
      <h1 class="display-4 fw-bold lh-1 text-body-emphasis mb-3">Final Score: </h1>
      <h1 class="display-4 fw-bold lh-1 text-body-emphasis mb-3">${score}</h1>
      <h2 id="outof" class='score'>${resultText}</h3>


    <!-- Tabs navs -->
<ul class="nav nav-tabs nav-justified mb-3" id="ex1" role="tablist">
  <li class="nav-item" role="presentation">
    <a
      class="nav-link active"
      id="score-tab-1"
      data-mdb-toggle="tab"
      href="#score-tabs-1"
      role="tab"
      aria-controls="ex3-tabs-1"
      aria-selected="true"
      >Your Progress</a
    >
  </li>
  <li class="nav-item" role="presentation">
    <a
      class="nav-link"
      id="score-tab-2"
      data-mdb-toggle="tab"
      href="#score-tabs-2"
      role="tab"
      aria-controls="ex3-tabs-2"
      aria-selected="false"
      >Top Score Board </a
    >
  </li>
</ul>


<!-- Tabs content -->
<div class="tab-content" id="ex2-content">
  <div
    class="tab-pane fade show active"
    id="score-tabs-1"
    role="tabpanel"
    aria-labelledby="score-tab-1"
  >
      <div id="chart" class="align-items-center"></div>
  </div>

  <div
    class="tab-pane fade"
    id="score-tabs-2"
    role="tabpanel"
    aria-labelledby="ex3-tab-3"
  >
      <div  id="top-scores" class="align-items-center"></div>
  </div>
</div>

      <div class = 'progress-buttons'>
        ${progress}
      <br>
      <div id = "show-answers">
      </div>
      <button class='start-btn btn btn-primary mt-4 mb-2' onclick='startNewGame()'>Play Again</button>
      <br/>
      <button class='start-btn btn btn-secondary' onclick='homeScreen()'>Home Screen</button>
      </div>
    </div>
    </div>`

  $("#game").fadeToggle(() => {
    gameContainer.innerHTML = gameHtml;
    addDataPoint(new Date().toLocaleString(), score);
    showChart()
    createTopBoard()
    $("#game").fadeToggle();
  });
  return;
}

function startNewGame() {
  $("#chartContainer").hide()
  $("#chartContainer").appendTo($("body"))
  startGame()
}

/**
 * show the progress chart if the user played at least 3 games
 */
function showChart() {
  if (chart.data.labels.length > 2) {
    var chartElem = $("#chartContainer")
    chartElem.show()
    chartElem.appendTo($("#chart"))
  }
  else {
    $("#outof").append(`\n<br>Play another ${3 - chart.data.labels.length} games to see the progress chart`)
  }
}


/**
 * this function creates a line chart to display the player's scores over time.
 */
function createLineChart(playerData) {
  // get only 15 last items
  if (!playerData) { playerData = [] }
  playerData = playerData.length < 16 ? playerData : playerData.slice(playerData.length - 14)
  // Get the chart container element
  const chartContainer = document.getElementById('chartContainer');

  let dates = []
  let scores = []
  playerData.forEach(element => {
    scores.push(element[1])
    dates.push(element[0])
  })

  if (chart) { chart.destroy(); }

  // Create the chart using Chart.js
  const ctx = chartContainer.getContext('2d');
  chart = new Chart(ctx, {
    type: 'line',
    data: {
      labels: dates,
      datasets: [{
        label: "Score",
        data: scores,
        borderColor: 'rgb(75, 192, 192)',
        fill: false
      }]
    },
    options: {
      responsive: true,
    }
  });
}

function createExampleChart() {
  // Get the chart container element
  const chartContainer = document.getElementById('exampleChart');

  // Create the chart using Chart.js
  const ctx = chartContainer.getContext('2d');
  chart = new Chart(ctx, {
    type: 'line',
    data: {
      labels: ['4 Scores Ago', '3 Scores Ago', '2 Scores Ago', 'Last Score', 'Current Score'],
      datasets: [{
        label: "Score",
        data: [0, 3, 2, 4, 5],
        borderColor: 'rgb(75, 192, 192)',
        fill: false
      }]
    },
    options: {
      responsive: true,
    }
  });
}

function addDataPoint(label, data) {
  chart.data.labels.push(label);
  chart.data.datasets[0].data.push(data);
  chart.update(); // Update the chart to reflect the changes
}

/**
 * Shuffles the elements in an array randomly.
 */
function shuffleArray(array) {
  for (let i = array.length - 1; i > 0; i--) {
    const j = Math.floor(Math.random() * (i + 1));
    [array[i], array[j]] = [array[j], array[i]];
  }
  return array;
}

/**
 * This function creates the table leaderboard of the top scores .
 */
function createTopBoard() {
  var scores = [];
  users.forEach(user => {
    try {
      let total = 0;
      let lastPlayed;
      user.scores.forEach(score => {
        total += score[1];
      })
      lastPlayed = user.scores[user.scores.length - 1][0];
      scores.push({ score: total, username: user.userName, date: lastPlayed })
    }
    catch (err) { }
  })
  scores.sort((a, b) => b.score - a.score);
  table = document.createElement('table')
  table.className = "table table-striped table-hover"
  table.innerHTML = `<thead><tr><th scope="col">Username</th><th scope="col">Total Score</th><th scope="col">Last Played</th></tr></thead>`
  scores = scores.slice(0, 5)
  scores.forEach(score => {
    table.innerHTML += `<tr class=""><td>${score.username}</td><td>${score.score}</td><td>${score.date}</td></tr>`
  })
  document.getElementById('top-scores').appendChild(table)
}

function checkAnswer(index, answer) {

  const correct = document.getElementById(answer);
  correct.className = correct.className.replace("btn-primary", "btn-success")

  if (index == answer) {
    userAnswers.push(true);
    score++;
  }
  else {
    userAnswers.push(false);
    const incorrect = document.getElementById(index);
    incorrect.className = incorrect.className.replace("btn-primary", "btn-danger")
  }
  currentQuestion++;
  setTimeout(function() {
    showQuestions();
  }, 1000);
}







Writing index.js


In [ ]:
from IPython.display import HTML
from jinja2 import Template


# open sytles.css
with open("styles.css", "r") as f:
  css_contents = f.read()

# open index.html
with open("index.html", "r") as f:
    html_contents = f.read()

# open index.js
with open("index.js", "r") as f:
    js_contents = f.read()


template = Template(html_contents)
html = template.render(styles=css_contents, script=js_contents)

display(HTML(html))


In [ ]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from firebase import firebase
from ipywidgets.widgets import widget
import ipywidgets as widgets
import IPython
from IPython.display import clear_output,display, HTML

# connect to firevase
FBconn = firebase.FirebaseApplication('https://tir6db-default-rtdb.firebaseio.com/',None)


text1=widgets.Text(description = "Question:")
text2=widgets.Text(description = "Option1:")
text3=widgets.Text(description ="Option2:")
text4=widgets.Text(description = "Option3:")
text5=widgets.Text(description = "Option4:")
text6=widgets.Text(description = "Correct:")

header = widgets.HTML(value=' <div style="font-size: 20px; font-weight: bold; "> Hello Admin</div>')
label_html = widgets.HTML(value=' <div style="font-size: 20px;"> Remove Question<br><p style="font-size: 12px; font-weight: bold; " >choose one of the values to remove it</p> </div>')



button = widgets.Button(description="Add to DataBase!")
button_Update = widgets.Button(description="Updaet Question")
button_Remove_Question = widgets.Button(description="Remove From DataBase!")
output = widgets.Output()

# Func that saves a question to a Firebase database, ensuring that all required fields are filled before the data is uploaded.
def save_question(b):
  data_to_upload={
     'Question' :  text1.value,
     'Option1' : text2.value,
     'Option2' : text3.value,
     'Option3' : text4.value,
     'Option4' : text5.value,
     'Correct' : text6.value
  }
  for key,val in data_to_upload.items():
    if(data_to_upload[key]==""):
      print("Cant Add Empty Field In Add Question Form!!")
      return

  result = FBconn.post('/Questions/', data_to_upload)
  with output:
    print("Question added: ID: ", result)
    # Get all documents in the 'questions' collection and store them in a list
  show_questions()



dropdown = widgets.Dropdown(options=[""], description='Select an option:', value="")

# This function displays the questions from the DB on the screen using widgets
def show_questions():
  global dropdown
  global questions_dict
  global question_titles
  result = FBconn.get('/Questions', None)
  html = '<img src="https://www.svgrepo.com/show/115092/bee.svg" alt="Bee Icon" width="100" height="100">'+'<h2>Questions</h2>\n';
    # Generate the HTML page with the list of questions
  questions_dict={}
  question_titles=[]
  for key in result:
    val =result[key]
    questions_dict[val["Question"]]=key
    question_titles.append(val["Question"])
    if (val['Question']==""):
        break
    html += '<ul>\n'
    html += f'<li>{val["Question"]}</li>\n'
    html += '</ul>'
  # Display the HTML page in the notebook
  dropdown = widgets.Dropdown(options=question_titles, description='Select an option:')
  dropdown.observe(dropdown_eventhandler)
  clear_output()
  display(header,HTML(html) ,dropdown ,text1,text2,text3,text4,text5,text6, output, button,button_Update,button_Remove_Question , label_html)
  # attach the function to the change event of the drop-down widget
  return questions_dict,question_titles


# This function is a help to display a question on the screen
def dropdown_eventhandler(change):
  global question_dict
  if change["type"] == "change":
    val = dropdown.value
    path = questions_dict[val]
    result = FBconn.get('/Questions', path)
    text1.value = result["Question"]
    text2.value = result["Option1"]
    text3.value = result["Option2"]
    text4.value = result["Option3"]
    text5.value = result["Option4"]
    text6.value = result["Correct"]

# function for updating an existing question in a database and changing one of its values
def update_question(event):
  global question_dict
  if (text1.value == "" or text2.value == "" or text3.value == "" or text4.value == "" or text5.value == "" or text6.value == ""):
    print("Cant Add Empty Field In Add Question Form!!")
    return
  val=dropdown.value
  path = questions_dict[val]
  FBconn.delete('/Questions/',path)
  save_question(event)
  print(f'Question {val} updated successfully.')

# A function to delete a question from the database
def remove_from_DB(event):
  global questions_dict
  if(len(dropdown.options)==1):
    print("Cant Delete All Questions! ,  Please Insert New Question Before")
    return
  print(questions_dict)
  val=dropdown.value
  path = questions_dict[val]

  # connect to firebase
  # define the path to the data you want to delete
  # call the delete method with the path as argument
  FBconn.delete('/Questions/',path)
  questions_dict,question_titles= show_questions()
  print(f'Question {val} deleted successfully.')


questions_dict,question_titles =show_questions()
# display the drop-down widget
button.on_click(save_question)
button_Update.on_click(update_question)
button_Remove_Question.on_click(remove_from_DB)



HTML(value=' <div style="font-size: 20px; font-weight: bold; "> Hello Admin</div>')

Dropdown(description='Select an option:', options=('Which keyword is used to define a function?', 'Which of th…

Text(value='', description='Question:')

Text(value='', description='Option1:')

Text(value='', description='Option2:')

...

Output()

Button(description='Add to DataBase!', style=ButtonStyle())

Button(description='Updaet Question', style=ButtonStyle())

Button(description='Remove From DataBase!', style=ButtonStyle())

HTML(value=' <div style="font-size: 20px;"> Remove Question<br><p style="font-size: 12px; font-weight: bold; "…